# マルコフモデル

まずはシンプルなモデルで言語モデルを学ぶ。

In [2]:
# import os; os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import random
from typing import Dict

import markovify
import tensorflow as tf
import tensorflow_datasets as tfds
import MeCab


---

## マルコフモデル

*Markov Model*

マルコフ過程に従う確率モデル。マルコフ連鎖とも。マルコフ過程とは、ある状態から次の状態へ遷移する確率が、その状態のみに依存する確率過程のこと。

$$
p(x_t | x_{t-1}, x_{t-2}, \ldots, x_1) = p(x_t | x_{t-1})
$$


---

## マルコフモデルを用いた言語モデル

マルコフモデルによる言語モデル（以後マルコフモデルと呼ぶ）では、状態を単語とし、ある単語の次に続く単語の確率を記述する。

自然な文章を生成するためには、前章で作ったようなランダムな出力を行うモデルではなく、入力された文脈を考慮したモデルが必要となる。マルコフモデルはそのようなモデルの中で特に理解しやすいシンプルなモデルである。このモデルは、与えられた文脈の中の最後の単語のみに着目し、次の単語を予測する。

マルコフモデルでは、モデルが持っている全ての語彙に対して、その次に続く単語とその確率を記述する。

例えば、モデルが以下の語彙を持っているとする。

- 今日
- カレー
- 天気
- おいしい
- は
- いい
- 。

これらに対して、次に続く単語とその確率を定める。適当に、主観で決めてみよう。

単語 | 次に続く単語(確率)
--- | ---
今日 | は(1.0)
カレー | は(1.0)
天気 | は(0.5), 。(0.5)
おいしい | 。(0.5), カレー(0.5)
は | 今日(0.25), カレー(0.25), おいしい(0.25), いい(0.25)
いい | 天気(0.5), 。(0.5)
。 |

句点は文の終わりを表すので、次に続く単語はないと考える。

辞書として定義しておこう。

In [2]:
vocab = {
    "今日": {"は": 1},
    "カレー": {"は": 1},
    "天気": {"は": 0.5, "。": 0.5},
    "おいしい": {"。": 0.5, "カレー": 0.5},
    "は": {"今日": 0.25, "カレー": 0.25, "おいしい": 0.25, "いい": 0.25},
    "いい": {"天気": 0.5, "。": 0.5},
}

マルコフモデルは、この確率に基づいて次の単語を予測する。それを繰り返して文章を生成する。

モデルを作ってみよう。

In [3]:
WordProb = Dict[str, float]

class MarkovModel:
    def __init__(self, vocab: Dict[str, WordProb]):
        self.vocab = vocab

    def __call__(self, context):
        last_word = context[-1]
        words, dist = zip(*self.vocab[last_word].items())
        word, = random.choices(words, weights=dist)
        return word


できた。これで文章を生成してみよう。

In [4]:
def generate_sentence(model: MarkovModel, start_word: str):
    context = [start_word]
    next_word = start_word
    while next_word != "。":
        next_word = model(context)
        context.append(next_word)
    return " ".join(context)

初めの単語はこちらで指定する必要がある。

In [5]:
model = MarkovModel(vocab)
print(generate_sentence(model, "今日"))
print(generate_sentence(model, "カレー"))

今日 は 今日 は おいしい 。
カレー は 今日 は いい 天気 は 今日 は カレー は カレー は 今日 は 今日 は おいしい 。


文脈を考慮したモデルが作成できた。次に続く単語に対してまあまあ妥当な確率を手動で設定したので、少しは自然さが見られるはず。


---

## マルコフモデルの学習

語彙と次の単語の確率をデータセットから自動で決定する。

語彙に紐づいている次の単語の確率が、マルコフモデルの全てを表していると見られる。  
前節ではこの確率を手動で定めたが、データを元に自動で設定する事を考える。

このような、モデルの動きを決める値（マルコフモデルの場合は次に続く単語の確率）はパラメータと呼ぶ。そして、適切なパラメータをデータセットから自動で求める事を**機械学習**（や学習）と呼ぶ。

ではやってみよう。

データセットとして、いくつかの単語列を用意する。そして、データセットに出現する全ての単語を対象に、その次に続く単語とその確率を記録する。こうすることでマルコフモデルが完成する。

確率は単語の出現頻度から定める。例として以下の3つの単語列をデータセットとして考えてみよう。

- 今日 は いい 天気 です 。
- 今日 は カレー を 食べ ました 。
- 私 は 今日 カレー を 食べ ました 。
- 私 は カレー が 好き です 。

このデータセットから、例えば、「今日」の次に来る可能性がある単語が「は」と「カレー」であると分かる。出現頻度を見ると、「は」が2回、「カレー」が1回出現しているため、「は」に2/3、「カレー」に1/3の確率を定義できる。このようにして、全ての単語について次に続く単語とその確率を定義する。

実際にやってみよう。

In [6]:
data = [
    "今日 は いい 天気 です 。",
    "今日 は カレー を 食べ ました 。",
    "私 は 今日 カレー を 食べ ました 。",
    "私 は カレー が 好き です 。",
]

In [7]:
vocab = {}
for sent in data:
    sent = sent.split()
    for w1, w2 in zip(sent[:-1], sent[1:]):
        if w1 not in vocab:
            vocab[w1] = {}
        if w2 not in vocab[w1]:
            vocab[w1][w2] = 0
        vocab[w1][w2] += 1

for w1 in vocab:
    total = sum(vocab[w1].values())
    for w2 in vocab[w1]:
        vocab[w1][w2] /= total

vocab

{'今日': {'は': 0.6666666666666666, 'カレー': 0.3333333333333333},
 'は': {'いい': 0.25, 'カレー': 0.5, '今日': 0.25},
 'いい': {'天気': 1.0},
 '天気': {'です': 1.0},
 'です': {'。': 1.0},
 'カレー': {'を': 0.6666666666666666, 'が': 0.3333333333333333},
 'を': {'食べ': 1.0},
 '食べ': {'ました': 1.0},
 'ました': {'。': 1.0},
 '私': {'は': 1.0},
 'が': {'好き': 1.0},
 '好き': {'です': 1.0}}

これで、マルコフモデルの学習が完了したことになる。このモデルでいくつか文章を生成してみよう。

In [8]:
model = MarkovModel(vocab)

for _ in range(10):
    start_word, = random.choices(["今日", "私"]) # 最初の単語はランダム
    print(generate_sentence(model, start_word))

今日 カレー を 食べ ました 。
今日 カレー を 食べ ました 。
今日 は カレー が 好き です 。
今日 は 今日 は いい 天気 です 。
今日 カレー を 食べ ました 。
私 は いい 天気 です 。
私 は 今日 は いい 天気 です 。
私 は 今日 は カレー が 好き です 。
私 は いい 天気 です 。
私 は いい 天気 です 。


ランダムなモデルよりは自然な文章が生成されているのではないだろうか。

これで、データセットからマルコフモデルを学習し、文章を生成することが出来た。

### markovify

マルコフモデルを用いて文章を生成するためのライブラリ。

- [jsvine/markovify: A simple, extensible Markov chain generator.](https://github.com/jsvine/markovify)

これを使うと簡単にマルコフモデルを用いた言語モデルを実装できる。

In [9]:
import markovify

In [10]:
data # 再掲

['今日 は いい 天気 です 。',
 '今日 は カレー を 食べ ました 。',
 '私 は 今日 カレー を 食べ ました 。',
 '私 は カレー が 好き です 。']

In [11]:
model = markovify.Text(data, state_size=1) # 学習

これで学習が完了した。文章を生成してみる。

In [12]:
for _ in range(10):
    sentence = model.make_sentence()
    print(sentence)

私 は 今日 は いい 天気 です 。
私 は 今日 は カレー が 好き です 。
None
None
私 は 今日 は 今日 カレー を 食べ ました 。
私 は 今日 は いい 天気 です 。
None
今日 は 今日 は 今日 は 今日 カレー を 食べ ました 。
今日 は 今日 は カレー が 好き です 。
None



---

## より大きなデータセットの活用

ここまで、私が用意した3つの短文をデータセットとして言語モデルを作成した。本節ではもう少し大きなデータセットを用いて言語モデルを作成する。

### wiki40b

Wikipediaの記事を集めたデータセット。Tensorflow Datasetsに収録されているものを使用する。

[wiki40b  |  TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/wiki40b?hl=en#wiki40bja)

In [1]:
# import os; os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2" # tensorflowのログを非表示
import tensorflow_datasets as tfds
import tensorflow as tf

2024-10-09 22:22:21.462195: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-09 22:22:21.464491: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-09 22:22:21.470701: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-09 22:22:21.481143: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-09 22:22:21.484081: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-09 22:22:21.492566: I tensorflow/core/platform/cpu_feature_gu

In [2]:
ds = tfds.load("wiki40b/ja", split="test", data_dir="data")
ds = list(ds.as_numpy_iterator())

2024-10-09 22:22:23.679943: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


/home/komiya/works/language-models/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Dl Completed...:  10%|█         | 2/20 [00:00<00:05,  3.02 file/s]2024-10-09 22:22:37.420462: E external/local_tsl/tsl/platform/cloud/curl_http_request.cc:610] The transmission  of request 0x7f22d4021a40 (URI: https://storage.googleapis.com/tfds-data/datasets%2Fwiki40b%2Fja%2F1.3.0%2Fwiki40b-train.tfrecord-00014-of-00016) has been stuck at 8534697 of 67108864 bytes for 18446744073709551614 seconds and will be aborted. CURL timing information: lookup time: 0.008875 (No error), connect time: 0.013485 (No error), pre-transfer time: 0.068009 (No error), start-transfer time: 0.444177 (No error)
2024-10-09 22:22:37.420518: E external/local_tsl/tsl/platform/cloud/curl_http_request.cc:610] The transmission  of request 0x7f2

辞書のリストが得られる。  
`ds: List[Dict[str, bytes]]`

In [15]:
ex = ds[0]
ex

{'text': b'\n_START_ARTICLE_\n\xe3\x83\x93\xe3\x83\xbc\xe3\x83\x88\xe3\x81\x9f\xe3\x81\x91\xe3\x81\x97\xe3\x81\xae\xe6\x95\x99\xe7\xa7\x91\xe6\x9b\xb8\xe3\x81\xab\xe8\xbc\x89\xe3\x82\x89\xe3\x81\xaa\xe3\x81\x84\xe6\x97\xa5\xe6\x9c\xac\xe4\xba\xba\xe3\x81\xae\xe8\xac\x8e\n_START_SECTION_\n\xe6\xa6\x82\xe8\xa6\x81\n_START_PARAGRAPH_\n\xe3\x80\x8c\xe6\x95\x99\xe7\xa7\x91\xe6\x9b\xb8\xe3\x81\xab\xe3\x81\xaf\xe6\xb1\xba\xe3\x81\x97\xe3\x81\xa6\xe8\xbc\x89\xe3\x82\x89\xe3\x81\xaa\xe3\x81\x84\xe3\x80\x8d\xe6\x97\xa5\xe6\x9c\xac\xe4\xba\xba\xe3\x81\xae\xe8\xac\x8e\xe3\x82\x84\xe3\x81\x97\xe3\x81\x8d\xe3\x81\x9f\xe3\x82\x8a\xe3\x82\x92\xe5\xa4\x9a\xe8\xa7\x92\xe7\x9a\x84\xe3\x81\xab\xe6\xa4\x9c\xe8\xa8\xbc\xe3\x81\x97\xe3\x80\x81\xe6\x97\xa5\xe6\x9c\xac\xe4\xba\xba\xe3\x81\xaeDNA\xe3\x82\x92\xe8\xa7\xa3\xe6\x98\x8e\xe3\x81\x99\xe3\x82\x8b\xe3\x80\x82_NEWLINE_\xe6\x96\xb0\xe6\x98\xa5\xe7\x95\xaa\xe7\xb5\x84\xe3\x81\xa8\xe3\x81\x97\xe3\x81\xa6\xe5\xae\x9a\xe6\x9c\x9f\xe7\x9a\x84\xe3\x81\xab\xe6\x

テキストの中身はこんな感じ

In [16]:
print(ex["text"].decode())


_START_ARTICLE_
ビートたけしの教科書に載らない日本人の謎
_START_SECTION_
概要
_START_PARAGRAPH_
「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。_NEWLINE_新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。


セクションごとにまとめる

In [17]:
data = []
for sample in ds:
    text = sample["text"].decode()
    sections = text.split("_START_SECTION_")
    for section in sections[1:]:
        sentence = section.split("_START_PARAGRAPH_")[1]
        sentence = sentence.replace("_NEWLINE_", "")
        sentence = sentence.replace("\n", "")
        data.append(sentence)

print("num of data:", len(data))
data[:3] # examples

num of data: 89698


['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。']

後の章でも使うので書き出しておく。

In [18]:
text_path = "data/jawiki.txt"
with open(text_path, "w") as f:
    f.write("\n".join(data))

### トークン化

テキストをトークンごとに分割する。トークンとはモデルが扱える最小単位のことで、例えば単語が該当する。本節でも単語を最小単位=トークンとして、文章を単語列に変換する。


英語の文章は単語間にスペースが入っているので、それで終わりである。特にこちらで行う処理はない。一方で日本語の文章はそうではないので、こちらで分割する必要がある。

日本語トークン化には形態素解析器を使用する。これは自然言語処理で形態素解析（品詞の分析）に使用するツール。色々な種類があるが、ここではMeCabを使用する。

In [19]:
import MeCab

In [20]:
tagger = MeCab.Tagger("-Owakati") # 出力形式を分かち書きに指定
result = tagger.parse("私は猫が好きです。")
result

'私 は 猫 が 好き です 。 \n'

これを使って、学習データの全てを単語ごとに分割する。

In [21]:
data_wakati = []
for sentence in data:
    data_wakati.append(tagger.parse(sentence).strip())

data_wakati[:3] # examples

['「 教科 書 に は 決して 載ら ない 」 日本 人 の 謎 や しきたり を 多角 的 に 検証 し 、 日本 人 の DNA を 解明 する 。 新春 番組 と し て 定期 的 に 放送 さ れ て おり 、 年末 の 午前 中 に 再 放送 さ れる の が 恒例 と なっ て いる 。',
 'ライブ ドア 社員 で あっ た 初代 代表 取締 役 社長 の 山名 真由 に よっ て 企業 内 起業 の 形 で 創業 。 2005 年 に 株式 会社 ライブ ドア から 分割 さ れ て 設立 。 かつて は ライブ ドア ホールディングス （ 現 ・ LDH ） の 子 会社 で あっ た が 、 ノン コア 事業 の 整理 に ともない 、 株式 会社 ゲオ （ 現 ： 株式 会社 ゲオ ホールディングス ） に 所有 する 全 株式 を 譲渡 し 、 同社 の 完全 子 会社 と なっ た 。 「 ぽす れん 」 「 ゲオ 宅配 レンタル 」 の オン ライン DVD ・ CD ・ コミック レンタル サービス 及び 「 GEO Online 」 と 「 ゲオ アプリ 」 の アプリ ・ ウェブサイト 運営 の 大きく 分け て 2 事業 を 展開 し て いる 。 以前 は DVD 販売 等 の E コマース サービス 「 ぽす れん ストア 」 、 動画 配信 コンテンツ 「 ぽす れん BB 」 や 電子 書籍 配信 サービス の 「 GEO ☆ Books 」 事業 も 行っ て い た 。 オン ライン DVD レンタル 事業 で は 会員 数 は 10 万 人 （ 2005 年 9 月 時点 ） 。 2006 年 5 月 より CD レンタル を 開始 。 同業 他社 に は 、 カルチュア ・ コンビニエンス ・ クラブ が 運営 する 『 TSUTAYA DISCAS 』 の ほか 、 DMM . com が 運営 する 『 DMM . com オン ライン DVD レンタル 』 が ある 。 過去 に は 「 Yahoo ! レンタル DVD 」 と 「 楽天 レンタル 」 の 運営 を 受託 し て い た 。',
 '2005 年 の 一 時期 、 東京 の ラジオ 局 、 InterFM で 、 「 堀江 社長 も 使っ

### 実践

markovifyを使ってマルコフモデルを学習させ、文章を生成する。

In [22]:
model = markovify.Text(data_wakati, state_size=1)

In [23]:
for _ in range(5):
    sentence = model.make_sentence().replace(" ", "")
    print(sentence)

郡西部にまで同一呼称方法で4進出に斜上、賀蘭部制度にも含めて木製ニシュトレイスランが実現して使われることに用いられる。北にできる。表題曲とした。通行量以上あった形態変化したためであった。
MCのポジションで割っては第48チームとし、実際による卵塊を受け、最新のノウハウを習得して大会を温存して色の築造当初は、そこである。容器に昇格とミュラ住民の掃討した。しかしブレスト間であった。特に2000年5試合に送電線が交わされたシーズン、なぜその日には肯定された以外ではテヘラン会談の活躍も可能に存在しているものの勢力がずきずきし、2013年8月ｰ10月18日の防塁に自由都市で展開させる。安井英雄伝説になった。2013年に引っ越している。
アドベンチャーの秀句やイヴ・オタヴィオ・ヒベイロも運搬するなどに自動化係数γ-18世紀に選ばれて1990年、フィルングルント東端・ギャップを得た。
ジェダイの評価を保ち、一方、メキシコへの悪性腫瘍の時期に高丸からなる。しかしながら身障者も大型種牡馬としたアクバルの対象地域で膝のスターターとしフレッシュオールアメリカンに囲まれ、UFCからかえった。配給をハザマには、リロングウェがその後、主力戦車駆逐艦は近江彦根東は可能としており、議会はラインの小説で御家にはロシア移民が低迷したものでは自身が使用時に、これは少ない。
赤穂国際A得点5m以上の罪との全空気弁花LaLaonlineにて全国委員会を有する地下茎を軍規違反の家はいた。7月下旬には任意に存在している。


語彙が増えたことで多様な文章が生成されるようになった。

ただ、文脈の最後の単語しか考慮できないので、文章全体での自然さを作り出すことは難しい。


---

## N-gram

単語の分割方法。N個の単語を1つの単位として文章を分割する。

1単語ごとに分割するやり方は1-gramと呼べる。1-gramだと直前の1単語しか考慮できないので自然な文章を生成するのが難しい。N-gramではそれがN個に拡張されるので、N-gramを採用することでより広い文脈を考慮できるようになる。

- 1-gram: 今日 / は / いい / 天気 / です / 。
- 2-gram: 今日 は / は いい / いい 天気 / 天気 です / です 。
- 3-gram: 今日 は いい / は いい 天気 / いい 天気 です / 天気 です 。

ちなみに、1-gramは*uni-gram*、2-gramは*bi-gram*、3-gramは*tri-gram*と呼ばれる。

N-gramにおけるマルコフモデルの学習はこれまでと同様に行うことが出来る。N個の単語を1つの単位として次に続く単語の確率分布を求める。以下のデータセット:

- 今日 は いい 天気 です 。
- 今日 は カレー を 食べ ました 。
- 私 は 今日 カレー を 食べ ました 。
- 私 は カレー が 好き です 。

から、2-gramの確率分布を求めると:

- 「今日 は」の次は「いい」が0.5、「カレー」が0.5
- 「は いい」の次は「天気」が1
- 「いい 天気」の次は「です」が1
- ...

て感じ。